        Prototyping notebook for predicting stock volaitility, prices, etc using extra data from web trends, news, etc. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('io.parquet.engine', 'pyarrow')
import numpy as np
import seaborn
from glob import glob
from matplotlib import pyplot as plt
import os
import gc, os
from getpass import getpass
from utils import *
from models import siamese_model_dense, triplet_loss, siamese_network, classify_single_input, prep_triplet_data, train_siamese, prep_tensor_ds, train_emotion_classifier
import transformers
from tqdm.notebook import tqdm
import gc
from sklearn.model_selection import train_test_split
import scipy
from transformers import AutoModelForSequenceClassification, AutoConfig
import torch
import torch.nn as nn
import torch.utils.data as data

cuda = torch.device("cuda")
cpu = torch.device("cpu")
seed=42
np.random.seed(seed)
torch.manual_seed(seed)
rng = np.random.default_rng(seed=seed) 
# if you don't need the API downloads, you can set this to False
use_api = False 

print('Available GPU memory:', available_mem(), 'GB')

In [ ]:
# After this first run, you will only need to enter the password to load the api keys.
if use_api:
    # Ask for input of password to save API keys
    password = getpass("Enter password to save/load API keys: ");
    kaggle_api_key = fernet_key_encryption(password, 'Kaggle');
    #td_ameritrade_api_key = fernet_key_encryption(password, 'TD_Ameritrade')
    #data_nasdaq_key = fernet_key_encryption(password, 'Nasdaq');
    del password;
    gc.collect();
    get_datasets(kaggle_api_key)

In [ ]:
search_terms = aquire_stock_search_terms('data/Stock/')

In [ ]:
classes, train_triplets, test_triplets, x_train, y_train, x_test, y_test = prep_triplet_data(MODEL=f"cardiffnlp/twitter-xlm-roberta-base-sentiment")
ds_train, ds_test = prep_tensor_ds( x_train, y_train, x_test, y_test)

siamese_network_model = siamese_network(classes)
siamese_network_model = siamese_network_model.to(cuda)


In [ ]:
siamese_model, history = train_siamese(train_triplets, test_triplets, siamese_network_model, epochs=1)
for param in siamese_network_model.base.parameters():
    param.requires_grad = False
for param in siamese_network_model.parameters():
    param.requires_grad = False

In [ ]:
model = classify_single_input(siamese_network_model)
model = model.to(cuda)
model, history = train_emotion_classifier(model, ds_train, ds_test, epochs=1)